# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key


In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
# M WALTON - CONVERT CITY TO TITLE CASE
city_data_df["City"] = city_data_df["City"].str.title()
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Keflavik,64.0049,-22.5624,9.24,100,40,3.09,IS,1688879221
1,1,Shinshiro,34.9000,137.5000,28.02,85,100,1.64,JP,1688879221
2,2,Fort Bragg,35.1390,-79.0060,24.00,90,0,0.00,US,1688878964
3,3,Thunder Bay,48.4001,-89.3168,10.18,82,63,1.55,CA,1688879222
4,4,Goiatuba,-18.0125,-49.3547,18.16,39,16,1.61,BR,1688879222


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
city_data_df["Lat"]
city_data_df["Lng"]
city_data_df["Humidity"]
Humidity_map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    color = "City"
    
)

#points.opts(color ="z",legend_position = "left")

# Display the map
Humidity_map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_temp = (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] ==0)
ideal_temp_df = city_data_df[ideal_temp]


# Drop any rows with null values - Code Ref W3 schools, data unknown
clean_ideal_temp_df = ideal_temp_df.dropna()

# Display sample data
clean_ideal_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Fort Bragg,35.1390,-79.0060,24.00,90,0,0.00,US,1688878964
18,18,Tazacorte,28.6290,-17.9293,21.54,73,0,3.09,ES,1688878901
30,30,Whitehorse,60.7161,-135.0538,25.57,24,0,1.32,CA,1688878812
32,32,Newman,37.3138,-121.0208,21.13,54,0,3.16,US,1688879230
125,125,Remire-Montjoly,4.9167,-52.2667,24.02,100,0,0.00,GF,1688878833
139,139,Puerto Quijarro,-17.7833,-57.7667,22.33,56,0,0.65,BO,1688879258
188,188,Maraba,-5.3686,-49.1178,24.10,100,0,0.00,BR,1688879260
221,221,Kolonia,50.9333,6.9500,24.42,71,0,1.03,DE,1688879283
226,226,Riverton,40.5219,-111.9391,26.27,38,0,3.09,US,1688879285
230,230,Belyy Yar,53.6039,91.3903,23.94,57,0,4.00,RU,1688878939


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity 
#Code Ref:  Zach, 2022
hotel_df = clean_ideal_temp_df[["City", "Country","Lat", "Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,Fort Bragg,US,35.1390,-79.0060,90,
18,Tazacorte,ES,28.6290,-17.9293,73,
30,Whitehorse,CA,60.7161,-135.0538,24,
32,Newman,US,37.3138,-121.0208,54,
125,Remire-Montjoly,GF,4.9167,-52.2667,100,
139,Puerto Quijarro,BO,-17.7833,-57.7667,56,
188,Maraba,BR,-5.3686,-49.1178,100,
221,Kolonia,DE,50.9333,6.9500,71,
226,Riverton,US,40.5219,-111.9391,38,
230,Belyy Yar,RU,53.6039,91.3903,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
limit = 10
radius = 10000
params = {
    "categories": "accommodation.hotel", 
    "apiKey":geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    city = row["City"]
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["text"] = f"{city}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results 
    # format to title case
    print(f"{hotel_df.loc[index, 'City'].title()} - nearest hotel: {hotel_df.loc[index, 'Hotel Name'].title()}")

# Display sample data
hotel_df

Starting hotel search
Fort Bragg - nearest hotel: Airborne Inn Lodging
Tazacorte - nearest hotel: App Leyma
Whitehorse - nearest hotel: Town & Mountain Hotel
Newman - nearest hotel: No Hotel Found
Remire-Montjoly - nearest hotel: Complexe Belova
Puerto Quijarro - nearest hotel: No Hotel Found
Maraba - nearest hotel: Portobelo
Kolonia - nearest hotel: Wasserturm Hotel Cologne
Riverton - nearest hotel: Ramada By Wyndham Draper
Belyy Yar - nearest hotel: No Hotel Found
Lionel Town - nearest hotel: No Hotel Found
Ouarzazat - nearest hotel: Fint
Tecklenburg - nearest hotel: Hotel Pension Budke
Grants - nearest hotel: Southwest Motel
Blackwater - nearest hotel: No Hotel Found
Boa Vista - nearest hotel: Aipana Plaza Hotel
Cullman - nearest hotel: Comfort Suites
Sirte - nearest hotel: فندق المدينة-سرت
Fort St. John - nearest hotel: Fort St. John Motor Inn
Westport - nearest hotel: Norwalk Inn And Conference Center
Grantsville - nearest hotel: No Hotel Found


,City,Country,Lat,Lng,Humidity,Hotel Name
2,Fort Bragg,US,35.1390,-79.0060,90,Airborne Inn Lodging
18,Tazacorte,ES,28.6290,-17.9293,73,App Leyma
30,Whitehorse,CA,60.7161,-135.0538,24,Town & Mountain Hotel
32,Newman,US,37.3138,-121.0208,54,No hotel found
125,Remire-Montjoly,GF,4.9167,-52.2667,100,Complexe Belova
139,Puerto Quijarro,BO,-17.7833,-57.7667,56,No hotel found
188,Maraba,BR,-5.3686,-49.1178,100,Portobelo
221,Kolonia,DE,50.9333,6.9500,71,Wasserturm Hotel Cologne
226,Riverton,US,40.5219,-111.9391,38,Ramada by Wyndham Draper
230,Belyy Yar,RU,53.6039,91.3903,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hotel_df["Lat"]
hotel_df["Lng"]
hotel_df["City"]
hotel_df["Hotel Name"]
hotel_map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=["City", "Humidity","Hotel Name","Country"],
    geo = True,
    tiles = "ESRI",
    size = 100,
    frame_width = 800,
    frame_height = 600,
    color = "yellow"
)

# Display the map
hotel_map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)